In [1]:
import pandas as pd

In [2]:
def decode(n: int, size=64):
    s = ["0"] * size
    s[size-n-1] = "1"
    bit_string = "".join(s)
    return hex(int(bit_string, 2))[2:].zfill(size // 4)

In [3]:
f = open("control_code.txt", 'r')
stuff = f.read()
pre_df = []
for i,line in enumerate(stuff.split("\n")):
    con = line.split(":")[0]
    con_hex = decode(i)
    con_desc = "".join(line.split(':')[1:])
    pre_df.append({
        "con": con,
        "con_hex":con_hex,
        "con_desc":con_desc
    })
df = pd.DataFrame(pre_df)

In [4]:
def hex_or(hex1: str, hex2: str):
    return hex(int(hex1, 16) | int(hex2, 16))[2:].zfill(len(hex1))

In [5]:
def get_control_or(df,*args):
    hex_fin = None
    for arg in args:

        out = df[df['con']==arg]['con_hex'].item()
        if hex_fin is None:
            hex_fin = out
        else:
            hex_fin = hex_or(hex_fin, out)
    return hex_fin

### FETCH CYCLE

In [6]:
class ControlRom:
    def __init__(self, addr_width=256, data_width=64, debug=True):
        n_nibbles = data_width // 4
        self.data = ['0' * n_nibbles] * addr_width
        self.debug = debug
    def __getitem__(self, index):
        return bytes.fromhex(self.data[index])
    
    def __setitem__(self, index, value):
        """
        index: hex integer i.e 0x0f
        value: string value
        """
        if type(index) is not int:
            index = int(index, base=16)
        if self.debug:
            print(value)
        self.data[index] = value
    
    def save_as_hex(self, fname='control_rom_try.hex'):
        with open(fname, 'wb') as f:
            for line in self.data:
                f.write(bytes.fromhex(line))
    
# Example usage
rom = ControlRom()
rom.save_as_hex()

In [7]:
rom = ControlRom() # sequencer
op_rom = ControlRom(data_width=8) # controller

## Programming the Sequencer

### FETCH CYCLE

In [8]:
rom[0x00] = get_control_or(df, 'PCw', 'MARe')
rom[0x01] = get_control_or(df, 'MARo','PCe')
rom[0x02] = get_control_or(df,'MARo', 'MDRe','MDRlr')
rom[0x03] = get_control_or(df,'MDRo', 'MDRob','IRe')


0000000000000018
0000000000000021
0000000000000920
0000002000001200


### EXECUTION CYCLE (OP code-> seq)

In [9]:
# LDA : OP = 0x3a
op_rom[0x3a] = '04'
rom[0x04] = get_control_or(df, 'PCw', 'MARe')
rom[0x05] = get_control_or(df, 'MARo','PCe')
rom[0x06] = get_control_or(df,'MARo', 'MDRe','MDRlr')
# load MAR with the fetched address
rom[0x07] = get_control_or(df,'MDRo', 'MDRob', 'MARe')
rom[0x08] = get_control_or(df, 'MARo','MDRe','MDRlr')
rom[0x09] = get_control_or(df,'MDRo', 'MDRob', 'Ae' )

04
0000000000000018
0000000000000021
0000000000000920
0000000000001210
0000000000000920
0000000002001200


In [10]:
op_rom.save_as_hex("controller_programs/op_rom.hex")
rom.save_as_hex("controller_programs/control_rom.hex")

In [11]:
# This is too slow we need to write a compiler

#

In [12]:
def compile_file_and_save(fpath, con_save_path, op_save_path):
    with open(fpath, 'r') as file:
        data = file.read()
        # turn it into lines
        lines = data.split("\n")
        # first lines are fetch cycles
        rom = ControlRom(debug=False) # sequencer
        op_rom = ControlRom(data_width=8, debug=False) # controller
        for line in lines:
            if len(line.strip())==0:
                continue
            ctype, rest = line.split("-")
            if ctype == 'CON':
                rest1, rest2 = rest.split("=")
                addr = rest1.strip()
                cons = rest2.strip().split(" ")
                rom[addr] = get_control_or(df, *cons)
            if ctype == 'OP':
                rest1, rest2 = rest.split("=")
                
                addr = rest1.strip()
                value = rest2.strip()

                op_rom[addr] = value.split("x")[1]
        rom.save_as_hex(con_save_path)
        op_rom.save_as_hex(op_save_path)

In [13]:
compile_file_and_save("controller_programs/control_mini_abstraction.txt", 'controller_programs/control_rom.hex', 'controller_programs/op_rom.hex')